# Importing Libraries/Packages & Datasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import time
import copy

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

torch.backends.cudnn.benchmark=True
proc = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.manual_seed(5703)
torch.manual_seed(5703)
np.random.seed(5703)
random.seed(5703)

In [ ]:
fulldata = pd.read_csv('/kaggle/input/ids-all-attacks/CICIDS_ALLATTACKS.csv')
fulldata['Label'].value_counts()

In [ ]:
# Drop rows with 'Benign' and 'Infilteration' labels
fulldata = fulldata[(fulldata['Label'] != 'Benign') & (fulldata['Label'] != 'Infilteration')]

# Display the value counts of the labels after dropping
print("\nAfter dropping:")
print(fulldata['Label'].value_counts())

In [ ]:
train_datas = {}
mixed_data = pd.DataFrame()
old=[]
test_data = pd.DataFrame()
for i in fulldata['Label'].unique():
    if fulldata[fulldata['Label'] == i].shape[0] > 100000:
        print(i)
        old.append(i)
        train_datas[i] = fulldata[fulldata['Label'] == i][:100000]
        mixed_data = pd.concat([mixed_data, fulldata[fulldata['Label'] == i][100000:110000]], axis=0)
        test_data = pd.concat([test_data, fulldata[fulldata['Label'] == i][110000:120000]], axis=0)

In [ ]:
new=["DDOS attack-HOIC-1","DDOS attack-HOIC-2","Bot-1","FTP-BruteForce-1","SSH-Bruteforce-1"]
train_datas["DDOS attack-HOIC-1"]=fulldata[fulldata['Label'] == "DDOS attack-HOIC"][120000:220000]
train_datas["DDOS attack-HOIC-2"]=fulldata[fulldata['Label'] == "DDOS attack-HOIC"][220000:320000]
train_datas["Bot-1"]=fulldata[fulldata['Label'] == "Bot"][120000:220000]
train_datas["FTP-BruteForce-1"]=fulldata[fulldata['Label'] == "FTP-BruteForce"][120000:]
train_datas["SSH-Bruteforce-1"]=fulldata[fulldata['Label'] == "SSH-Bruteforce"][120000:160000]

"""mixed_data = pd.DataFrame()
for i in train_datas.keys():
    if i[:-2] in fulldata['Label'].unique():
        mixed_data = pd.concat([mixed_data, fulldata[fulldata['Label'] == i[:-2]][110000:120000]], axis=0)"""

In [ ]:
del fulldata

In [ ]:
mixed_data = mixed_data.sample(frac=1).reset_index(drop=True)
mixed_data_labels = list(mixed_data['Label'])
mixed_data.drop(['Label'], axis=1, inplace=True)
mixed_data = mixed_data.to_numpy()

test_data = test_data.sample(frac=1).reset_index(drop=True)
test_data_labels = list(test_data['Label'])
test_data.drop(['Label'], axis=1, inplace=True)
test_data = test_data.to_numpy()


for i in train_datas:
    train_datas[i].drop(['Label'], axis=1, inplace=True)
    #test_datas[i].drop(['Label'], axis=1, inplace=True)
    train_datas[i] = train_datas[i].sample(frac=1).reset_index(drop=True)
    #test_datas[i] = test_datas[i].sample(frac=1).reset_index(drop=True)
    train_datas[i] = train_datas[i].to_numpy()
    #test_datas[i] = test_datas[i].to_numpy()

In [ ]:
#loading data into pytorch dataloader as train and test
devices = []
batch_size = 32
train_loader = {}
for i in train_datas:
    train_loader[i] = torch.utils.data.DataLoader(train_datas[i], batch_size = batch_size, shuffle=True)
    devices.append(i)

In [ ]:
train_loader.keys()

In [ ]:
for label in train_datas.keys():
    print(len(train_datas[label]),train_datas[label].shape)

In [ ]:
#initialising config variables
num_clients = len(train_loader)     
num_selected = len(train_loader)    
num_rounds = 1     
epochs = 10        


## Deep Auto-encoder Client Models (Training and Testing)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

input_dim = train_loader['Bot'].dataset.shape[1]

# Define AEModel with regularization
class AEModel(nn.Module):
    def __init__(self, input_dim):
        super(AEModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 64)  
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 16)
        self.fc8 = nn.Linear(16, 24)
        self.fc9 = nn.Linear(24, 32)
        self.fc10 = nn.Linear(32, 48)
        self.fc11 = nn.Linear(48, 64)
        self.fc12 = nn.Linear(64, input_dim)
        self.fc13 = nn.Linear(input_dim, input_dim)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with probability 0.2
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.dropout(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc8(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc9(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc10(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc11(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc12(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc13(x)
        return x

In [ ]:
# Function for client update with regularization
def client_update(client_model, optimizer, train_data, epoch=3):
    client_model.train()
    criterion = nn.MSELoss(reduction='mean')
    for e in range(epoch):
        running_loss = 0.0
        for data in train_data:
            output = client_model(data.float())
            optimizer.zero_grad()
            loss = criterion(data.float().to(proc), output)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss / len(train_data)
    return epoch_loss

In [ ]:
client_models = {i: AEModel(input_dim).to(proc) for i in devices}
#opt = {i: optim.SGD(client_models[i].parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5) for i in devices}


In [ ]:
opt = {i: optim.Adam( client_models[i].parameters(), lr=0.0005,  weight_decay=1e-5) for i in devices}

In [ ]:
# Train Client Model and Global Model
start_time = time.time()
for r in range(num_rounds):  # total number of rounds
    print('\nround: ', r+1)
    loss = 0
    for i in devices:
        l = client_update(client_models[i], opt[i], train_loader[i], epochs)
        print('client: ', i, 'loss: ', l)
        loss += l
time_required = time.time() - start_time
print('/nTIME: {}mins'.format(time_required/60))

In [ ]:
import os
try:
    os.makedirs('/kaggle/working/models_extra/clients/')
except:
    pass
for i in devices:
    torch.save(client_models[i],'/kaggle/working/models_extra/clients/' + i + '.pt')

In [ ]:
client_models = {}
for device in old:
    client_models[device] = torch.load('/kaggle/input/models_fixed_overfit/pytorch/extramodels_levi_50epoch/1/models_extra_Levi/clients_Levi/' + device + '.pt')
client_models.keys()

In [ ]:
def quan_thresh(model, dataloader, quantile=0.9):
    model.eval()
    se = []
    for batch in dataloader:
        for data in batch:
            error = np.power(data.float().cpu().numpy() - model(data.float()).cpu().detach().numpy(), 2) # len(error) = 66
            se.append(sum(error))
    # Calculate threshold as quantile of errors 
    #thresh = np.quantile(se, quantile, axis=0) #before it was thresh = np.quantile(mse, quantile)
    #errs.append(thresh)
    #thresh = np.quantile(mse, quantile)
    return np.mean(se) + np.std(se), np.quantile(se, quantile), se #before it was np.mean(errs)
  #return thresh

def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_pred)):
        if y_actual[i] == y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_actual[i] != y_pred[i]:
            FP += 1
        if y_actual[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_actual[i] != y_pred[i]:
            FN += 1
    return (TP, FP, TN, FN)

In [ ]:
thresholds = {}
quans = {}
errors = {}
for device in devices:
    model = client_models[device]
    model.eval()
    dataloader = train_loader[device]
    thresholds[device], quans[device], errors[device] = quan_thresh(model, dataloader)
    #thresholds[devices[i]] = quan_recon(model, dataloader)
print(thresholds)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, rankdata, pearsonr
from sklearn.metrics import mutual_info_score
selectedattackbenmix_stats = []
res=[]
device_idx = {device: i for i, device in enumerate(devices)}
for device in devices:
    model = client_models[device]
    model.eval()
    labels = mixed_data_labels
    threshold = thresholds[device]
    temp=[]
    y_true = []
    y_pred = []
    ind = 0
    for data in mixed_data:
        data = torch.Tensor(data)
        error = np.sum(np.power(data.detach().numpy() - model(data).detach().numpy(), 2))
        temp.append(error)
        if device == labels[ind]:
            y_true.append(1)
            y_pred.append(1 if error < threshold else 0)
        else:
            y_true.append(0)
            y_pred.append(0 if error >= threshold else 1)
        ind += 1
    res.append([device,min(temp),max(temp)])
    TP, FP, TN, FN = perf_measure(y_true, y_pred)
    TP += 1
    FN += 1
    TN += 1
    FP += 1
    conf_matrix = [[TP, FN], [FP, TN]]
    plt.figure() 
    sns.heatmap(conf_matrix, annot=True, fmt="d")
    plt.title("Confusion Matrix for Client " + str(device) +" with mixed data")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

    acc = (TP+TN) / (TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1score = 2 * ((precision * recall) / (precision + recall))
    TPR = round((TP / (TP+FN)), 6)
    FPR = round((FP / (FP + TN)), 6)
    selectedattackbenmix_stats.append([str("Stats for Client " + str(device) +" with mixed data"),acc*100,precision,recall,F1score*100,TPR,FPR])

  #   [['TP', 'FN']
  #   ['FP', 'TN']]

In [ ]:
for i in selectedattackbenmix_stats:
    print("Title: " + i[0])
    print("Accuracy: " + str(i[1]))
    print("Precision: " + str(i[2]))
    print("Recall: " + str(i[3]))
    print("F1 score : " + str(i[4]))
    print("TPR : " + str(i[5]))
    print("FPR : " + str(i[6]))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

input_dim = train_loader['Bot'].dataset.shape[1]

# Define AEModel with regularization
class AEModel(nn.Module):
    def __init__(self, input_dim):
        super(AEModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 64)  
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 16)
        self.fc8 = nn.Linear(16, 24)
        self.fc9 = nn.Linear(24, 32)
        self.fc10 = nn.Linear(32, 48)
        self.fc11 = nn.Linear(48, 64)
        self.fc12 = nn.Linear(64, input_dim)
        self.fc13 = nn.Linear(input_dim, input_dim)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with probability 0.2
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.dropout(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc8(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc9(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc10(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc11(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc12(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc13(x)
        return x

In [ ]:
client_models = {}
for device in devices:
    client_models[device] = torch.load('/kaggle/input/models_fixed_overfit/pytorch/extramodels_levi_50epoch/1/models_extra_Levi/clients_Levi/' + device + '.pt')

In [ ]:
# Define the encoder-only model class
class AEModelEncoder(nn.Module):
    def __init__(self, input_dim):
        super(AEModelEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 64)  
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 16)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with probability 0.2
        
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.dropout(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.dropout(x)
        return x

# Function to load encoder weights from the full model
def load_encoder_weights(full_model, encoder_model):
    encoder_model.fc1.weight.data = full_model.fc1.weight.data
    encoder_model.fc1.bias.data = full_model.fc1.bias.data
    encoder_model.fc2.weight.data = full_model.fc2.weight.data
    encoder_model.fc2.bias.data = full_model.fc2.bias.data
    encoder_model.fc3.weight.data = full_model.fc3.weight.data
    encoder_model.fc3.bias.data = full_model.fc3.bias.data
    encoder_model.fc4.weight.data = full_model.fc4.weight.data
    encoder_model.fc4.bias.data = full_model.fc4.bias.data
    encoder_model.fc5.weight.data = full_model.fc5.weight.data
    encoder_model.fc5.bias.data = full_model.fc5.bias.data
    encoder_model.fc6.weight.data = full_model.fc6.weight.data
    encoder_model.fc6.bias.data = full_model.fc6.bias.data
    encoder_model.fc7.weight.data = full_model.fc7.weight.data
    encoder_model.fc7.bias.data = full_model.fc7.bias.data

# Update client_models to use the encoder-only model
client_models_encoders = {}
for device in devices:
    full_model = torch.load('/kaggle/input/models_fixed_overfit/pytorch/extramodels_levi_50epoch/1/models_extra_Levi/clients_Levi/' + device + '.pt')
    encoder_model = AEModelEncoder(input_dim)
    load_encoder_weights(full_model, encoder_model)
    client_models_encoders[device] = encoder_model

In [ ]:
agg_models={}
client_lens={}
agg_models_labels={}
for i in client_models_encoders.keys():
    if i[:-2] in client_models_encoders.keys():
        try:
            agg_models[i[:-2]].append(client_models_encoders[i])
            agg_models_labels[i[:-2]].append(i)
            client_lens[i[:-2]].append(len(train_datas[i]))
            print(i)
        except:
            print(i)
            agg_models[i[:-2]]=[client_models_encoders[i[:-2]],client_models_encoders[i]]
            agg_models_labels[i[:-2]]=[i[:-2],i]
            client_lens[i[:-2]]=[len(train_datas[i[:-2]]),len(train_datas[i])]
            

In [ ]:
# aggregates the model weights received from every client
# and updates the global model with updated weights
# FedAvg
def server_aggregate(global_model, client_models, client_lens):
    total = sum(client_lens)
    n = len(client_models)
    # n = num_selected
    global_dict = global_model.state_dict()
    for k in global_dict.keys(): # calculate average weight/bias --> avg_w/b
        global_dict[k] -= torch.stack([client_models[i].state_dict()[k].float() * (n * client_lens[i] / total) for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict()) # local model get updated weight/bias
        
# FedAvgM
def server_aggregate_M(global_model, client_models, client_lens):
    total = sum(client_lens)    # 592    sum [51, 122, 162, 257]
    n = len(client_models)      # 4 local clients
    global_dict = global_model.state_dict() # weight/bias dict --> {'encoder.0.weight': Tensor with shape torch.Size([86, 115]), 'encoder.0.bias':....} 16 items
    temp = copy.deepcopy(global_dict)       # temporary weight/bias dict
    v = {x:1 for x in copy.deepcopy(global_dict)}   # initialise v

    for i,k in enumerate(global_dict.keys()):
        # calculate average weight/bias --> avg_w/b
        temp[k] = torch.stack([client_models[i].state_dict()[k].float() * (n * client_lens[i] / total) for i in range(len(client_models))], 0).mean(0)
        temp_v = 0.9 * v[k] + temp[k]               # v = 0.9v + avg_w/b   momentum=0.9
        global_dict[k] = global_dict[k] - temp_v    # w = w - v
    global_model.load_state_dict(global_dict)

    for model in client_models:
        model.load_state_dict(global_model.state_dict()) # local model get updated weight/bias


def server_aggregate_fedprox(global_model, client_models, mu=0.05):
    n = len(client_models)
    global_dict = global_model.state_dict()
    
    for k in global_dict.keys():
        # Calculate the average of the client model parameters
        global_weight = sum(client_models[i].state_dict()[k].float() for i in range(n)) / n
        
        # Apply the FedProx proximal term
        prox_term = mu * global_dict[k]
        global_dict[k] = (global_weight + prox_term) / (1 + mu)
    
    # Load the updated parameters into the global model
    global_model.load_state_dict(global_dict)
    
    # Update each client model with the new global parameters
    for model in client_models:
        model.load_state_dict(global_model.state_dict())


In [ ]:
"""epochs=1
trained=[]
for r in range(2):
    print('\n Round: ', r+1)
    for i in agg_models.keys():
        server_aggregate_M(client_models_encoders[i],agg_models[i],client_lens[i])
        for j in client_models.keys():
            if j[:-2] in client_models.keys():
                load_encoder_weights(client_models_encoders[i],client_models[j])
                if j[:-2] not in trained:
                    load_encoder_weights(client_models_encoders[i],client_models[j[:-2]])
                    loss = 0
                    l = client_update(client_models[j[:-2]], opt[j[:-2]], train_loader[j[:-2]], epochs)
                    print('client: ', j[:-2], 'loss: ', l)
                    loss += l
                    trained.append(j[:-2])
                loss = 0
                l = client_update(client_models[j], opt[j], train_loader[j], epochs)
                print('client: ', j, 'loss: ', l)
                loss += l
        trained=[]
        for k in agg_models[i]:
            for j in client_models.keys():
                if agg_models_labels[i][agg_models[i].index(k)] == j:
                    load_encoder_weights(client_models[j],k)"""

In [ ]:
"""epochs=2
trained=[]
for r in range(5):
    print('\nround: ', r+1)
    for i in agg_models.keys():
        server_aggregate_fedprox(client_models_encoders[i],agg_models[i])
        for k in agg_models[i]:
            for j in client_models_encoders.keys():
                if agg_models_labels[i][agg_models[i].index(k)] == j:
                    load_encoder_weights(client_models_encoders[j],client_models[j])
    for j in client_models.keys():
        if j[:-2] in client_models.keys():
            if j[:-2] not in trained:
                for name, param in client_models[j].named_parameters():
                    if 'fc7' in name or 'fc8' in name or 'fc9' in name or 'fc10' in name or 'fc11' in name or 'fc12' in name or 'fc13' in name:
                        param.requires_grad = False
                loss = 0
                l = client_update(client_models[j[:-2]], opt[j[:-2]], train_loader[j[:-2]], epochs)
                print('client: ', j[:-2], 'loss: ', l)
                loss += l
                trained.append(j[:-2])
            loss = 0
            for name, param in client_models[j].named_parameters():
                if 'fc7' in name or 'fc8' in name or 'fc9' in name or 'fc10' in name or 'fc11' in name or 'fc12' in name or 'fc13' in name:
                    param.requires_grad = False
            l = client_update(client_models[j], opt[j], train_loader[j], epochs)
            print('client: ', j, 'loss: ', l)
            loss += l
    trained=[]
    for i in agg_models.keys():
        for k in agg_models[i]:
            for j in client_models.keys():
                if agg_models_labels[i][agg_models[i].index(k)] == j:
                    load_encoder_weights(client_models[j],k)
        

for i in agg_models.keys():
    server_aggregate_fedprox(client_models_encoders[i],agg_models[i])
    for k in agg_models[i]:
        for j in client_models_encoders.keys():
            if agg_models_labels[i][agg_models[i].index(k)] == j:
                load_encoder_weights(client_models_encoders[j],client_models[j])"""

In [ ]:
#opt = {i: optim.Adam( client_models[i].parameters(), lr=0.0005,  weight_decay=1e-5) for i in devices}

In [ ]:
opt = {i: optim.SGD(client_models[i].parameters(), lr=0.0005, momentum=0.9, weight_decay=1e-5) for i in devices}

In [ ]:
epochs=2
trained=[]
for r in range(5):
    print('\nround: ', r+1)
    for i in agg_models.keys():
        server_aggregate_fedprox(client_models_encoders[i],agg_models[i])
        for k in agg_models[i]:
            for j in client_models_encoders.keys():
                if agg_models_labels[i][agg_models[i].index(k)] == j:
                    load_encoder_weights(client_models_encoders[j],client_models[j])
    for j in client_models.keys():
        if j[:-2] in client_models.keys():
            if j[:-2] not in trained:
                loss = 0
                l = client_update(client_models[j[:-2]], opt[j[:-2]], train_loader[j[:-2]], epochs)
                print('client: ', j[:-2], 'loss: ', l)
                loss += l
                trained.append(j[:-2])
            loss = 0
            l = client_update(client_models[j], opt[j], train_loader[j], epochs)
            print('client: ', j, 'loss: ', l)
            loss += l
    trained=[]
    for i in agg_models.keys():
        for k in agg_models[i]:
            for j in client_models.keys():
                if agg_models_labels[i][agg_models[i].index(k)] == j:
                    load_encoder_weights(client_models[j],k)
        

for i in agg_models.keys():
    server_aggregate_fedprox(client_models_encoders[i],agg_models[i])
    for k in agg_models[i]:
        for j in client_models_encoders.keys():
            if agg_models_labels[i][agg_models[i].index(k)] == j:
                load_encoder_weights(client_models_encoders[j],client_models[j])

In [ ]:
import os
try:
    os.makedirs('/kaggle/working/models_extra/clients/')
except:
    pass
for i in devices:
    torch.save(client_models[i],'/kaggle/working/models_extra/clients/' + i + '.pt')

## Global model creation (K-means)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

input_dim = train_loader['Bot'].dataset.shape[1]

# Define AEModel with regularization
class AEModel(nn.Module):
    def __init__(self, input_dim):
        super(AEModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 64)  
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 16)
        self.fc8 = nn.Linear(16, 24)
        self.fc9 = nn.Linear(24, 32)
        self.fc10 = nn.Linear(32, 48)
        self.fc11 = nn.Linear(48, 64)
        self.fc12 = nn.Linear(64, input_dim)
        self.fc13 = nn.Linear(input_dim, input_dim)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with probability 0.2
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.dropout(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc8(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc9(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc10(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc11(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc12(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc13(x)
        return x

In [ ]:
"""client_models = {}
for device in devices:
    client_models[device] = torch.load('/kaggle/input/models_fixed_overfit/pytorch/extramodels/1/' + device + '.pt')"""

In [ ]:
# Define the encoder-only model class
class AEModelEncoder(nn.Module):
    def __init__(self, input_dim):
        super(AEModelEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 64)  
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 16)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with probability 0.2
        
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.dropout(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.dropout(x)
        return x

# Function to load encoder weights from the full model
def load_encoder_weights(full_model, encoder_model):
    encoder_model.fc1.weight.data = full_model.fc1.weight.data
    encoder_model.fc1.bias.data = full_model.fc1.bias.data
    encoder_model.fc2.weight.data = full_model.fc2.weight.data
    encoder_model.fc2.bias.data = full_model.fc2.bias.data
    encoder_model.fc3.weight.data = full_model.fc3.weight.data
    encoder_model.fc3.bias.data = full_model.fc3.bias.data
    encoder_model.fc4.weight.data = full_model.fc4.weight.data
    encoder_model.fc4.bias.data = full_model.fc4.bias.data
    encoder_model.fc5.weight.data = full_model.fc5.weight.data
    encoder_model.fc5.bias.data = full_model.fc5.bias.data
    encoder_model.fc6.weight.data = full_model.fc6.weight.data
    encoder_model.fc6.bias.data = full_model.fc6.bias.data
    encoder_model.fc7.weight.data = full_model.fc7.weight.data
    encoder_model.fc7.bias.data = full_model.fc7.bias.data

# Update client_models to use the encoder-only model
client_models = {}
for device in old:
    full_model = torch.load('/kaggle/input/models_fixed_overfit/pytorch/extramodels_levi_50epoch/1/models_extra_Levi/clients_Levi/' + device + '.pt')
    encoder_model = AEModelEncoder(input_dim)
    load_encoder_weights(full_model, encoder_model)
    client_models[device] = encoder_model
client_models.keys()

In [ ]:
compressed_outs = {}
compressed_outs_test = {}
for i in client_models:
    model = client_models[i]
    model.eval()
    dataloader = train_loader[i]
    outs = []
    ind = 0
    flag = False
    for batch in dataloader:
        for data in batch:
            layer_output = model(data.float()).detach().numpy()
            outs.append(layer_output)
            ind += 1
            if ind == 20000:
                compressed_outs[i] = outs
                outs = []
            elif ind == 20010:
                compressed_outs_test[i] = outs
                flag = True
                break
        if flag:
            break

In [ ]:
for i in compressed_outs:
    if i[:-2] in compressed_outs:
        for j in compressed_outs[i]:
            compressed_outs[i[:-2]].append(j)

In [ ]:
l=[]
for i in compressed_outs:
    if i[-2]=="-":
        l.append(i)
for i in l:
    compressed_outs.pop(i)

In [ ]:
for i in compressed_outs.keys():
    print(i,': ',len(compressed_outs[i]))


In [ ]:
"""import random

# Function to shuffle and reduce the number of samples for each label
def shuffle_and_reduce(data_dict, num_samples=10000):
    reduced_data_dict = {}
    for label, data in data_dict.items():
        # Shuffle the data
        random.shuffle(data)
        # Store only the specified number of samples
        reduced_data_dict[label] = data[:num_samples]
    return reduced_data_dict
# Apply the shuffle and reduce function
compressed_outs_reduced = shuffle_and_reduce(compressed_outs, num_samples=10000)

# Print the results to verify
for label, data in compressed_outs_reduced.items():
    print(f"{label} : {len(data)}")"""

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import numpy as np

# Extract data from the dictionary
data = []
labels = []

for label, tensors in compressed_outs.items():
    for tensor in tensors:
        data.append(tensor)
        labels.append(label)

data = np.array(data)
labels = np.array(labels)

# Get the number of samples
num_samples = len(data)

# Generate a random permutation of indices
permutation_indices = np.random.permutation(num_samples)

# Shuffle data and labels arrays using the permutation
shuffled_data = data[permutation_indices]
shuffled_labels = labels[permutation_indices]
data=shuffled_data
labels=shuffled_labels

In [ ]:
"""from joblib import load

kmeans = load('//kaggle/input/models_fixed_overfit/other/kmeans/1/kmeans_model.joblib')"""

In [ ]:
n=22
# Apply KMeans clustering
kmeans = KMeans(n_clusters=n,random_state=100)

In [ ]:
cluster_labels = kmeans.fit_predict(data)

# Get the cluster centroids
centroids = kmeans.cluster_centers_

# Create a larger figure
fig, ax = plt.subplots(figsize=(10, 8))

# Visualize the clusters with different colors for each label
scatter = ax.scatter(data[:, 0], data[:, 1], c=cluster_labels, cmap='Set1', alpha=0.5)

# Plot the centroids
ax.scatter(centroids[:, 0], centroids[:, 1], c='yellowgreen', s=200, alpha=0.5)

# Add legend with label names only if there are valid elements
if scatter.legend_elements()[0] is not None and len(scatter.legend_elements()[0]) > 0:
    unique_labels = list(set(labels))
    legend_labels = [f'{label} ({i})' for i, label in enumerate(unique_labels)]
    ax.legend(handles=scatter.legend_elements()[0], labels=legend_labels, title='Labels')

ax.set_title('KMeans Clustering')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')

plt.tight_layout()

plt.show()

In [ ]:
# Dictionary to store attacks and their counts for each cluster
cluster_attacks_count = {cluster: {} for cluster in range(n)}

# Iterate through data and cluster labels
for idx, cluster_label in enumerate(cluster_labels):
    attack = labels[idx]
    if attack not in cluster_attacks_count[cluster_label]:
        cluster_attacks_count[cluster_label][attack] = 1
        
    else:
        cluster_attacks_count[cluster_label][attack] += 1
    
# Print attacks and their counts for each cluster
for cluster, attack_counts in cluster_attacks_count.items():
    print(f"Cluster {cluster}: {attack_counts}")

In [ ]:
cluster_labels = {}
for i in range(n):
    cluster_labels[i] = max(cluster_attacks_count[i], key=cluster_attacks_count[i].get)

In [ ]:
cluster_labels={0: 'Bot',
 1: 'DDOS attack-HOIC',
 2: 'FTP-BruteForce',
 3: 'DoS attacks-Hulk',
 4: 'DoS attacks-SlowHTTPTest',
 5: 'DDoS attacks-LOIC-HTTP',
 6: 'DDoS attacks-LOIC-HTTP',
 7: 'SSH-Bruteforce',
 8: 'DDOS attack-HOIC',
 9: 'SSH-Bruteforce',
 10: 'Bot',
 11: 'DoS attacks-Hulk',
 12: 'FTP-BruteForce',
 13: 'DoS attacks-SlowHTTPTest',
 14: 'DoS attacks-Hulk',
 15: 'FTP-BruteForce',
 16: 'DoS attacks-SlowHTTPTest',
 17: 'FTP-BruteForce',
 18: 'DoS attacks-SlowHTTPTest',
 19: 'DDoS attacks-LOIC-HTTP',
 20: 'DoS attacks-SlowHTTPTest',
 21: 'DDoS attacks-LOIC-HTTP'}

In [ ]:
l=[]
for i in client_models.keys():
    if i[-2]=="-":
        l.append(i)
for i in l:
    client_models.pop(i)

In [ ]:
compressed_mixed = {}
j=0
for data in mixed_data:
    if j<1000:
        for i in client_models:
            model = client_models[i]
            model.eval()
            layer_output = model(torch.Tensor(data))
            try:
                compressed_mixed[i].append(layer_output)
            except:
                compressed_mixed[i] = [layer_output]
    j+=1

In [ ]:
"""#working
n=27
threshold = 0.016"""

In [ ]:
old_thresholds={'Bot': 0.31418854651204847,
 'DDOS attack-HOIC': 0.03866640203996175,
 'DDoS attacks-LOIC-HTTP': 0.46889474594200486,
 'DoS attacks-Hulk': 0.48591240802465874,
 'DoS attacks-SlowHTTPTest': 0.1051760870436359,
 'FTP-BruteForce': 0.04449623069596497,
 'SSH-Bruteforce': 0.07784218174722095}

In [ ]:
def classify_new_data(new_data_point, threshold):
    # Predict the cluster for the new data point
    new_data_cluster = kmeans.predict(np.array(new_data_point))
    # Get the centroid of the predicted cluster
    centroid = kmeans.cluster_centers_[new_data_cluster]
    # Calculate the Euclidean distance between the new data point and the centroid
    distance = np.sqrt(np.sum((new_data_point - centroid) ** 2))
    # Classify based on the distance
    if distance < threshold:
        return new_data_cluster[0]
    else:
        return False

num_encoders=7
num_clusters=22
num_clients=7
met=[]
start = 0
end =  min(old_thresholds.values())*num_encoders
step = end/num_clusters
i = start
first_round=[]
prev_TPR=0

for x in range(num_clients):
    met=[]
    print("Round :"+str(x+1))
    while i < end:
        TP,FP,TN,FN = 0,0,0,0
        # Iterate through the compressed_mixed dictionary and classify each data point
        for model, data_list in compressed_mixed.items():
            j = 0
            for data in data_list:
                # Convert the data to NumPy array and detach from computation graph
                data_np = data.detach().numpy()
                data_np = data_np.reshape(1,-1)
                # Convert the data to double data type before passing it to KMeans model
                if classify_new_data(data_np, i) == False:
                    classification_result = False
                else: 
                    classification_result = cluster_labels[classify_new_data(data_np, i)]
                if classification_result == False:
                    if model == mixed_data_labels[j]:
                        FN += 1
                    else:
                        TN += 1
                else:
                    if classification_result == mixed_data_labels[j]:
                        if model == mixed_data_labels[j]:
                            TP += 1
                        else:
                            FP += 1
                    else:
                        FN += 1
                j += 1

        if TP==0:
            TP=0.1
        # Calculate accuracy
        accuracy = (TP + TN) / (TP + TN + FP + FN)

        # Calculate true positive rate (TPR) or sensitivity or recall
        TPR = TP / (TP + FN)

        # Calculate false positive rate (FPR)
        FPR = FP / (FP + TN)

        # Calculate specificity or true negative rate (TNR)
        TNR = TN / (FP + TN)

        # Calculate precision or positive predictive value (PPV)
        PPV = TP / (TP + FP)

        met.append([i,accuracy,TPR,FPR,TNR])
        if x==0:
            first_round.append([i,accuracy,TPR,FPR,TNR])
        i += step
    print(met)
    # Extracting data from 'fed_met' list
    thresholds = [entry[0] for entry in met]
    TPR = [entry[2] for entry in met]

    # Find the best thresholds
    # best_accuracy_idx = np.argmax(accuracy)

    best_tpr_idx = np.argmax(TPR)
    best_TPR = round(TPR[best_tpr_idx], 4)
    if best_tpr_idx==0:
        start=thresholds[best_tpr_idx]
    else:
        start=thresholds[best_tpr_idx-1]
    if best_tpr_idx == len(thresholds) - 1:
        end = thresholds[best_tpr_idx]
    else:
        end = thresholds[best_tpr_idx + 1]
    step = end/num_clusters
    i=start
    if prev_TPR==best_TPR:
        break
    prev_TPR=best_TPR
    #nxt_acc_thresh = threshold
    #nxt_acc_thresh = thresholds[best_accuracy_idx+1]
    #nxt_tpr_thresh = thresholds[best_tpr_idx+1]


In [ ]:
# Extracting data from 'fed_met' list
thresholds = [entry[0] for entry in met]
accuracy = [entry[1] for entry in met]
TPR = [entry[2] for entry in met]
FPR = [entry[3] for entry in met]
TNR = [entry[4] for entry in met]

# Find the best thresholds
best_accuracy_idx = np.argmax(accuracy)
best_tpr_idx = np.argmax(TPR)
best_fpr_idx = np.argmin(FPR)

best_accuracy_threshold = thresholds[best_accuracy_idx]
best_tpr_threshold = thresholds[best_tpr_idx]
best_fpr_threshold = thresholds[best_fpr_idx]

best_tpr_threshold

In [ ]:
# Extracting data from 'fed_met' list
thresholds = [entry[0] for entry in first_round]
accuracy = [entry[1] for entry in first_round]
TPR = [entry[2] for entry in first_round]
FPR = [entry[3] for entry in first_round]
TNR = [entry[4] for entry in first_round]

# Find the best thresholds
best_accuracy_idx = np.argmax(accuracy)
best_tpr_idx = np.argmax(TPR)
best_fpr_idx = np.argmin(FPR)

best_accuracy_threshold = thresholds[best_accuracy_idx]
best_tpr_threshold = thresholds[best_tpr_idx]
best_fpr_threshold = thresholds[best_fpr_idx]

# Plotting
plt.figure(figsize=(14, 8))

# Accuracy
plt.plot(thresholds, accuracy, label='Accuracy', marker='o')
plt.scatter(best_accuracy_threshold, accuracy[best_accuracy_idx], color='red')  # Highlight best accuracy

# True Positive Rate (TPR)
plt.plot(thresholds, TPR, label='True Positive Rate (TPR)', marker='o')
plt.scatter(best_tpr_threshold, TPR[best_tpr_idx], color='green')  # Highlight best TPR

# False Positive Rate (FPR)
plt.plot(thresholds, FPR, label='False Positive Rate (FPR)', marker='o')
plt.scatter(best_fpr_threshold, FPR[best_fpr_idx], color='blue')  # Highlight best (lowest) FPR
# True Negative Rate (TNR)
plt.plot(thresholds, TNR, label='True Negative Rate (TNR)', marker='o')

# Adjusting x-axis scale
plt.xlim(min(thresholds), max(thresholds))
plt.xticks(np.linspace(min(thresholds), max(thresholds), num=10))

plt.xlabel('Threshold')
plt.ylabel('Metric Value')
plt.title('Classification Metrics vs. Threshold (Global Classifier)')
plt.legend(loc=(0.76, 0.67))
plt.grid(True)

# Add vertical dashed lines and annotate the x-axis
plt.axvline(x=best_accuracy_threshold, color='red', linestyle='--')
plt.annotate(f'{best_accuracy_threshold:.5f}', xy=(best_accuracy_threshold, 0), xytext=(best_accuracy_threshold, -0.05),
             arrowprops=dict(facecolor='red', shrink=0.05), horizontalalignment='center')

plt.axvline(x=best_tpr_threshold, color='green', linestyle='--')
plt.annotate(f'{best_tpr_threshold:.5f}', xy=(best_tpr_threshold, 0), xytext=(best_tpr_threshold, -0.05),
             arrowprops=dict(facecolor='green', shrink=0.05), horizontalalignment='center')

"""# Add a text box with the best values
textstr = '\n'.join((
    f'Best Accuracy: {accuracy[best_accuracy_idx]:.5f} at threshold {best_accuracy_threshold}',
    f'Best TPR: {TPR[best_tpr_idx]:.5f} at threshold {best_tpr_threshold}',
))

# Place a text box to the right of the plot
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
plt.gcf().text(0.95, 0.5, textstr, fontsize=12, verticalalignment='center', bbox=props)

plt.show()"""
print(f'\nBest Accuracy: {accuracy[best_accuracy_idx]:.5f} at threshold {best_accuracy_threshold}',
    f'\nBest TPR: {TPR[best_tpr_idx]:.5f} at threshold {best_tpr_threshold}\n') 

In [ ]:
compressed_test = {}
j=0
for data in test_data:
    for i in client_models:
        model = client_models[i]
        model.eval()
        layer_output = model(torch.Tensor(data))
        try:
            compressed_test[i].append(layer_output)
        except:
            compressed_test[i] = [layer_output]

In [ ]:
best_accuracy_threshold=0.04250108653979267

In [ ]:
def classify_new_data(new_data_point, threshold):
    # Predict the cluster for the new data point
    new_data_cluster = kmeans.predict(np.array(new_data_point))
    # Get the centroid of the predicted cluster
    centroid = kmeans.cluster_centers_[new_data_cluster]
    # Calculate the Euclidean distance between the new data point and the centroid
    distance = np.sqrt(np.sum((new_data_point - centroid) ** 2))
    # Classify based on the distance'
    
    if distance < threshold:
        return new_data_cluster[0]
    else:
        return False

threshold = best_tpr_threshold



# Iterate through the compressed_mixed dictionary and classify each data point
for model, data_list in compressed_test.items():
    TP,FP,TN,FN = 0, 0, 0, 0
    j = 0
    for data in data_list:
        # Convert the data to NumPy array and detach from computation graph
        data_np = data.detach().numpy()
        data_np = data_np.reshape(1,-1)
        # Convert the data to double data type before passing it to KMeans model
        if classify_new_data(data_np, threshold) == False:
            classification_result = False
        else: 
            classification_result = cluster_labels[classify_new_data(data_np, threshold)]
        if classification_result == False:
            if model == test_data_labels[j]:
                FN += 1
            else:
                TN += 1
        else:
            if classification_result == test_data_labels[j]:
                if model == test_data_labels[j]:
                    TP += 1
                else:
                    FP += 1
            else:
                FN += 1
        j += 1
            


    print()
    # Calculate accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    # Calculate true positive rate (TPR) or sensitivity or recall
    TPR = TP / (TP + FN)

    # Calculate false positive rate (FPR)
    FPR = FP / (FP + TN)

    # Calculate specificity or true negative rate (TNR)
    TNR = TN / (FP + TN)

    # Calculate precision or positive predictive value (PPV)
    PPV = TP / (TP + FP)



    # Print the metrics with up to four decimal places
    print(f"Data from :" +str(model))
    print("TP:", f"{TP}")
    print("TN:", f"{TN}")
    print("FP:", f"{FP}")
    print("FN:", f"{FN}")
    print("Accuracy:", f"{accuracy*100:.4f}")
    print("True Positive Rate (TPR):", f"{TPR:.4f}")
    print("False Positive Rate (FPR):", f"{FPR:.4f}")
    print("True Negative Rate (TNR):", f"{TNR:.4f}")
    print("Positive Predictive Value (PPV):", f"{PPV:.4f}")
    print()

In [ ]:

threshold = best_tpr_threshold
TP,FP,TN,FN = 0,0,0,0

# Iterate through the compressed_mixed dictionary and classify each data point
for model, data_list in compressed_test.items():
    j = 0
    for data in data_list:
        # Convert the data to NumPy array and detach from computation graph
        data_np = data.detach().numpy()
        data_np = data_np.reshape(1,-1)
        # Convert the data to double data type before passing it to KMeans model
        if classify_new_data(data_np, threshold) == False:
            classification_result = False
        else: 
            classification_result = cluster_labels[classify_new_data(data_np, threshold)]
        if classification_result == False:
            if model == test_data_labels[j]:
                FN += 1
            else:
                TN += 1
        else:
            if classification_result == test_data_labels[j]:
                if model == test_data_labels[j]:
                    TP += 1
                else:
                    FP += 1
            else:
                FN += 1
        j += 1
            


print()
# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate true positive rate (TPR) or sensitivity or recall
TPR = TP / (TP + FN)

# Calculate false positive rate (FPR)
FPR = FP / (FP + TN)

# Calculate specificity or true negative rate (TNR)
TNR = TN / (FP + TN)

# Calculate precision or positive predictive value (PPV)
PPV = TP / (TP + FP)



# Print the metrics with up to four decimal places
print(f"Global Classifier :")
print("TP:", f"{TP}")
print("TN:", f"{TN}")
print("FP:", f"{FP}")
print("FN:", f"{FN}")
print("Accuracy:", f"{accuracy*100:.4f}")
print("True Positive Rate (TPR):", f"{TPR:.4f}")
print("False Positive Rate (FPR):", f"{FPR:.4f}")
print("True Negative Rate (TNR):", f"{TNR:.4f}")
print("Positive Predictive Value (PPV):", f"{PPV:.4f}")
print()

In [ ]:
from collections import Counter
c=Counter(mixed_data_labels)
c

In [ ]:
import os
from joblib import dump

# Define the directory path
directory = '/kaggle/working/models_fixed_overfit'

# Create the directory if it does not exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the kmeans model
dump(kmeans, '/kaggle/working/models_fixed_overfit/kmeans_model.joblib')


Adding clusters

In [ ]:
labels_to_remove=[]
for i in new:
    if i[:-2] in old and i[:-2] not in labels_to_remove:
        labels_to_remove.append(i[:-2])

In [ ]:
labels_to_remove

In [ ]:
# Define the encoder-only model class
class AEModelEncoder(nn.Module):
    def __init__(self, input_dim):
        super(AEModelEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 64)  
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 16)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with probability 0.2
        
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.dropout(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.dropout(x)
        return x

# Function to load encoder weights from the full model
def load_encoder_weights(full_model, encoder_model):
    encoder_model.fc1.weight.data = full_model.fc1.weight.data
    encoder_model.fc1.bias.data = full_model.fc1.bias.data
    encoder_model.fc2.weight.data = full_model.fc2.weight.data
    encoder_model.fc2.bias.data = full_model.fc2.bias.data
    encoder_model.fc3.weight.data = full_model.fc3.weight.data
    encoder_model.fc3.bias.data = full_model.fc3.bias.data
    encoder_model.fc4.weight.data = full_model.fc4.weight.data
    encoder_model.fc4.bias.data = full_model.fc4.bias.data
    encoder_model.fc5.weight.data = full_model.fc5.weight.data
    encoder_model.fc5.bias.data = full_model.fc5.bias.data
    encoder_model.fc6.weight.data = full_model.fc6.weight.data
    encoder_model.fc6.bias.data = full_model.fc6.bias.data
    encoder_model.fc7.weight.data = full_model.fc7.weight.data
    encoder_model.fc7.bias.data = full_model.fc7.bias.data

# Update client_models to use the encoder-only model
extra_clients = {}
for device in devices:
    if device[:-2] in devices or device in labels_to_remove:
        full_model = torch.load('/kaggle/input/models_fixed_overfit/pytorch/sgd_fed_harlem/1/models_extra/clients/' + device + '.pt')
        encoder_model = AEModelEncoder(input_dim)
        load_encoder_weights(full_model, encoder_model)
        extra_clients[device] = encoder_model
extra_clients.keys()

In [ ]:
extra_compressed_outs = {}
extra_compressed_outs_test = {}
for i in extra_clients:
    model = extra_clients[i]
    model.eval()
    dataloader = train_loader[i]
    outs = []
    ind = 0
    flag = False
    for batch in dataloader:
        for data in batch:
            layer_output = model(data.float()).detach().numpy()
            outs.append(layer_output)
            ind += 1
            if ind == 10000:
                extra_compressed_outs[i] = outs
                outs = []
            elif ind == 10010:
                extra_compressed_outs_test[i] = outs
                flag = True
                break
        if flag:
            break

In [ ]:
for i in extra_compressed_outs:
    if i[:-2] in extra_compressed_outs:
        for j in extra_compressed_outs[i]:
            extra_compressed_outs[i[:-2]].append(j)

In [ ]:
l=[]
for i in extra_compressed_outs:
    if i[-2]=="-":
        l.append(i)
for i in l:
    extra_compressed_outs.pop(i)

In [ ]:
for i in extra_compressed_outs.keys():
    print(i,': ',len(extra_compressed_outs[i]))


In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import numpy as np

# Extract data from the dictionary
data_new = []
labels_new = []

for label, tensors in extra_compressed_outs.items():
    for tensor in tensors:
        data_new.append(tensor)
        labels_new.append(label)

data_new = np.array(data_new)
labels_new = np.array(labels_new)

# Get the number of samples
num_samples = len(data_new)

# Generate a random permutation of indices
permutation_indices = np.random.permutation(num_samples)

# Shuffle data and labels arrays using the permutation
shuffled_data = data_new[permutation_indices]
shuffled_labels = labels_new[permutation_indices]
data_new=shuffled_data
labels_new=shuffled_labels

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Assuming you have the trained KMeans model stored as kmeans_model
# Assuming you have the cluster labels stored in cluster_labels dictionary
# Assuming you have the labels you want to remove stored in labels_to_remove list

centroids_to_keep = []
labels_to_keep = []

# Iterate over the cluster labels
for cluster, label in cluster_labels.items():
    # Check if the label is not in labels_to_remove
    if label not in labels_to_remove:
        # Append the cluster centroid and label for clusters to keep
        centroids_to_keep.append(kmeans.cluster_centers_[cluster])
        labels_to_keep.append(label)

# Convert the lists to numpy arrays
centroids_to_keep = np.array(centroids_to_keep)
labels_to_keep = np.array(labels_to_keep)

In [ ]:
print(len(centroids_to_keep))
print(n)
print(n-len(centroids_to_keep))

In [ ]:
labels_to_keep

In [ ]:
# Store the previous centroids
previous_centroids = centroids_to_keep

# Calculate the total number of clusters for the new model
new_n_clusters = n-len(centroids_to_keep)

# Create a new KMeans model with the desired number of clusters
new_kmeans = KMeans(n_clusters=new_n_clusters, random_state=100)
new_kmeans.fit(data_new)
# Initialize the centroids of the new model with the previous centroids
new_kmeans.cluster_centers_ = np.vstack((previous_centroids, new_kmeans.cluster_centers_))

In [ ]:
cluster_labels = new_kmeans.predict(data_new)
# Dictionary to store attacks and their counts for each cluster
cluster_attacks_count = {cluster: {} for cluster in range(len(new_kmeans.cluster_centers_))}

# Iterate through data and cluster labels
for idx, cluster_label in enumerate(cluster_labels):
    attack = labels_new[idx]
    if attack not in cluster_attacks_count[cluster_label]:
        cluster_attacks_count[cluster_label][attack] = 1
        
    else:
        cluster_attacks_count[cluster_label][attack] += 1
# Print attacks and their counts for each cluster
for cluster, attack_counts in cluster_attacks_count.items():
    print(f"Cluster {cluster}: {attack_counts}")

In [ ]:
for cluster, attack_counts in cluster_attacks_count.items():
    if cluster<len(labels_to_keep):
        cluster_attacks_count[cluster]={labels_to_keep[cluster] : 1}
cluster_attacks_count

In [ ]:
# Define the encoder-only model class
class AEModelEncoder(nn.Module):
    def __init__(self, input_dim):
        super(AEModelEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 64)  
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 16)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with probability 0.2
        
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.dropout(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.dropout(x)
        return x

# Function to load encoder weights from the full model
def load_encoder_weights(full_model, encoder_model):
    encoder_model.fc1.weight.data = full_model.fc1.weight.data
    encoder_model.fc1.bias.data = full_model.fc1.bias.data
    encoder_model.fc2.weight.data = full_model.fc2.weight.data
    encoder_model.fc2.bias.data = full_model.fc2.bias.data
    encoder_model.fc3.weight.data = full_model.fc3.weight.data
    encoder_model.fc3.bias.data = full_model.fc3.bias.data
    encoder_model.fc4.weight.data = full_model.fc4.weight.data
    encoder_model.fc4.bias.data = full_model.fc4.bias.data
    encoder_model.fc5.weight.data = full_model.fc5.weight.data
    encoder_model.fc5.bias.data = full_model.fc5.bias.data
    encoder_model.fc6.weight.data = full_model.fc6.weight.data
    encoder_model.fc6.bias.data = full_model.fc6.bias.data
    encoder_model.fc7.weight.data = full_model.fc7.weight.data
    encoder_model.fc7.bias.data = full_model.fc7.bias.data

# Update client_models to use the encoder-only model
client_models = {}
for device in devices:
    full_model = torch.load('/kaggle/input/models_fixed_overfit/pytorch/sgd_fed_harlem/1/models_extra/clients/' + device + '.pt')
    encoder_model = AEModelEncoder(input_dim)
    load_encoder_weights(full_model, encoder_model)
    client_models[device] = encoder_model
client_models.keys()

In [ ]:
"""compressed_outs = {}
compressed_outs_test = {}
for i in client_models:
    model = client_models[i]
    model.eval()
    dataloader = train_loader[i]
    outs = []
    ind = 0
    flag = False
    for batch in dataloader:
        for data in batch:
            layer_output = model(data.float()).detach().numpy()
            outs.append(layer_output)
            ind += 1
            if ind == 10000:
                compressed_outs[i] = outs
                outs = []
            elif ind == 10010:
                compressed_outs_test[i] = outs
                flag = True
                break
        if flag:
            break"""

In [ ]:
"""for i in compressed_outs:
    if i[:-2] in compressed_outs:
        for j in compressed_outs[i]:
            compressed_outs[i[:-2]].append(j)"""

In [ ]:
"""l=[]
for i in compressed_outs:
    if i[-2]=="-":
        l.append(i)
for i in l:
    compressed_outs.pop(i)"""

In [ ]:
"""for i in compressed_outs.keys():
    print(i,': ',len(compressed_outs[i]))"""

In [ ]:
"""import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import numpy as np

# Extract data from the dictionary
data = []
labels = []

for label, tensors in compressed_outs.items():
    for tensor in tensors:
        data.append(tensor)
        labels.append(label)

data = np.array(data)
labels = np.array(labels)

# Get the number of samples
num_samples = len(data)

# Generate a random permutation of indices
permutation_indices = np.random.permutation(num_samples)

# Shuffle data and labels arrays using the permutation
shuffled_data = data[permutation_indices]
shuffled_labels = labels[permutation_indices]
data=shuffled_data
labels=shuffled_labels"""

In [ ]:
"""cluster_labels = new_kmeans.predict(data)

# Get the cluster centroids
centroids = new_kmeans.cluster_centers_"""

In [ ]:
"""# Dictionary to store attacks and their counts for each cluster
n=len(centroids)
cluster_attacks_count = {cluster: {} for cluster in range(n)}

# Iterate through data and cluster labels
for idx, cluster_label in enumerate(cluster_labels):
    attack = labels[idx]
    if attack not in cluster_attacks_count[cluster_label]:
        cluster_attacks_count[cluster_label][attack] = 1
        
    else:
        cluster_attacks_count[cluster_label][attack] += 1
    
# Print attacks and their counts for each cluster
for cluster, attack_counts in cluster_attacks_count.items():
    print(f"Cluster {cluster}: {attack_counts}")"""

In [ ]:
cluster_labels = {}
for i in range(len(cluster_attacks_count)):
    cluster_labels[i] = max(cluster_attacks_count[i], key=cluster_attacks_count[i].get)
cluster_labels

In [ ]:
cluster_labels={0: 'DoS attacks-Hulk',
 1: 'DoS attacks-SlowHTTPTest',
 2: 'DDoS attacks-LOIC-HTTP',
 3: 'DDoS attacks-LOIC-HTTP',
 4: 'DoS attacks-Hulk',
 5: 'DoS attacks-SlowHTTPTest',
 6: 'DoS attacks-Hulk',
 7: 'DoS attacks-SlowHTTPTest',
 8: 'DoS attacks-SlowHTTPTest',
 9: 'DDoS attacks-LOIC-HTTP',
 10: 'DoS attacks-SlowHTTPTest',
 11: 'DDoS attacks-LOIC-HTTP',
 12: 'FTP-BruteForce',
 13: 'DDOS attack-HOIC',
 14: 'DDOS attack-HOIC',
 15: 'FTP-BruteForce',
 16: 'SSH-Bruteforce',
 17: 'SSH-Bruteforce',
 18: 'Bot',
 19: 'Bot',
 20: 'FTP-BruteForce',
 21: 'FTP-BruteForce'}

In [ ]:
l=[]
for i in client_models.keys():
    if i[-2]=="-":
        l.append(i)
for i in l:
    client_models.pop(i)

In [ ]:
compressed_mixed = {}
j=0
for data in mixed_data:
    if j<1000:
        for i in client_models:
            model = client_models[i]
            model.eval()
            layer_output = model(torch.Tensor(data))
            try:
                compressed_mixed[i].append(layer_output)
            except:
                compressed_mixed[i] = [layer_output]
    j+=1

In [ ]:
old_thresholds={'Bot': 0.31418854651204847,
 'DDOS attack-HOIC': 0.03866640203996175,
 'DDoS attacks-LOIC-HTTP': 0.46889474594200486,
 'DoS attacks-Hulk': 0.48591240802465874,
 'DoS attacks-SlowHTTPTest': 0.1051760870436359,
 'FTP-BruteForce': 0.04449623069596497,
 'SSH-Bruteforce': 0.07784218174722095}

In [ ]:
def classify_new_data_fed(new_data_point, threshold):
    # Predict the cluster for the new data point
    new_data_cluster = new_kmeans.predict(np.array(new_data_point))
    # Get the centroid of the predicted cluster
    centroid = new_kmeans.cluster_centers_[new_data_cluster]
    # Calculate the Euclidean distance between the new data point and the centroid
    distance = np.sqrt(np.sum((new_data_point - centroid) ** 2))
    # Classify based on the distance
    if distance < threshold:
        return new_data_cluster[0]
    else:
        return False


num_encoders=7
num_clusters=22
num_clients=12
fed_met=[]
start = 0
end =  min(old_thresholds.values())*num_encoders
step = end/num_clusters
i = start
first_fed_round=[]
prev_TPR=0

for x in range(num_clients):
    fed_met=[]
    print("Round :"+str(x+1))
    while i < end:
        TP,FP,TN,FN = 0,0,0,0
        # Iterate through the compressed_mixed dictionary and classify each data point
        for model, data_list in compressed_mixed.items():
            j = 0
            for data in data_list:
                # Convert the data to NumPy array and detach from computation graph
                data_np = data.detach().numpy()
                data_np = data_np.reshape(1,-1)
                # Convert the data to double data type before passing it to KMeans model
                if classify_new_data_fed(data_np, i) == False:
                    classification_result = False
                else: 
                    classification_result = cluster_labels[classify_new_data_fed(data_np, i)]
                if classification_result == False:
                    if model == mixed_data_labels[j]:
                        FN += 1
                    else:
                        TN += 1
                else:
                    if classification_result == mixed_data_labels[j]:
                        if model == mixed_data_labels[j]:
                            TP += 1
                        else:
                            FP += 1
                    else:
                        FN += 1
                j += 1

        if TP==0:
            TP=0.1

        #print(i)
        # Calculate accuracy
        accuracy = (TP + TN) / (TP + TN + FP + FN)

        # Calculate true positive rate (TPR) or sensitivity or recall
        TPR = TP / (TP + FN)

        # Calculate false positive rate (FPR)
        FPR = FP / (FP + TN)

        # Calculate specificity or true negative rate (TNR)
        TNR = TN / (FP + TN)

        # Calculate precision or positive predictive value (PPV)
        PPV = TP / (TP + FP)

        fed_met.append([i,accuracy,TPR,FPR,TNR])
        if x==0:
            first_fed_round.append([i,accuracy,TPR,FPR,TNR])
        i += step
    print(fed_met)
    # Extracting data from 'fed_met' list
    thresholds = [entry[0] for entry in fed_met]
    TPR = [entry[2] for entry in fed_met]

    # Find the best thresholds
    # best_accuracy_idx = np.argmax(accuracy)

    best_tpr_idx = np.argmax(TPR)
    best_TPR = round(TPR[best_tpr_idx], 4)
    if best_tpr_idx==0:
        start=thresholds[best_tpr_idx]
    else:
        start=thresholds[best_tpr_idx-1]
    if best_tpr_idx == len(thresholds) - 1:
        end = thresholds[best_tpr_idx]
    else:
        end = thresholds[best_tpr_idx + 1]
    step = end/num_clusters
    i=start
    if prev_TPR==best_TPR:
        break
    prev_TPR=best_TPR
    #nxt_acc_thresh = thresholds[best_accuracy_idx+1]
    #nxt_tpr_thresh = thresholds[best_tpr_idx+1]


In [ ]:
# Extracting data from 'fed_met' list
thresholds = [entry[0] for entry in fed_met]
accuracy = [entry[1] for entry in fed_met]
TPR = [entry[2] for entry in fed_met]
FPR = [entry[3] for entry in fed_met]
TNR = [entry[4] for entry in fed_met]

# Find the best thresholds
best_accuracy_idx = np.argmax(accuracy)
best_tpr_idx = np.argmax(TPR)
best_fpr_idx = np.argmin(FPR)

best_accuracy_threshold = thresholds[best_accuracy_idx]
best_tpr_threshold = thresholds[best_tpr_idx]
best_tpr_threshold

In [ ]:
# Extracting data from 'fed_met' list
thresholds = [entry[0] for entry in first_fed_round]
accuracy = [entry[1] for entry in first_fed_round]
TPR = [entry[2] for entry in first_fed_round]
FPR = [entry[3] for entry in first_fed_round]
TNR = [entry[4] for entry in first_fed_round]

# Find the best thresholds
best_accuracy_idx = np.argmax(accuracy)
best_tpr_idx = np.argmax(TPR)
best_fpr_idx = np.argmin(FPR)

best_accuracy_threshold = thresholds[best_accuracy_idx]
best_tpr_threshold = thresholds[best_tpr_idx]

# Plotting
plt.figure(figsize=(14, 8))

# Accuracy
plt.plot(thresholds, accuracy, label='Accuracy', marker='o')
plt.scatter(best_accuracy_threshold, accuracy[best_accuracy_idx], color='red')  # Highlight best accuracy

# True Positive Rate (TPR)
plt.plot(thresholds, TPR, label='True Positive Rate (TPR)', marker='o')
plt.scatter(best_tpr_threshold, TPR[best_tpr_idx], color='green')  # Highlight best TPR

# False Positive Rate (FPR)
plt.plot(thresholds, FPR, label='False Positive Rate (FPR)', marker='o')
plt.scatter(best_fpr_threshold, FPR[best_fpr_idx], color='blue')  # Highlight best (lowest) FPR
# True Negative Rate (TNR)
plt.plot(thresholds, TNR, label='True Negative Rate (TNR)', marker='o')

# Adjusting x-axis scale
plt.xlim(min(thresholds), max(thresholds))
plt.xticks(np.linspace(min(thresholds), max(thresholds), num=10))

plt.xlabel('Threshold')
plt.ylabel('Metric Value')
plt.title('Classification Metrics vs. Threshold (Global Classifier)')
plt.legend(loc=(0.75, 0.6))
plt.grid(True)

# Add vertical dashed lines and annotate the x-axis
plt.axvline(x=best_accuracy_threshold, color='red', linestyle='--')
plt.annotate(f'{best_accuracy_threshold:.5f}', xy=(best_accuracy_threshold, 0), xytext=(best_accuracy_threshold, -0.05),
             arrowprops=dict(facecolor='red', shrink=0.05), horizontalalignment='center')

plt.axvline(x=best_tpr_threshold, color='green', linestyle='--')
plt.annotate(f'{best_tpr_threshold:.5f}', xy=(best_tpr_threshold, 0), xytext=(best_tpr_threshold, -0.05),
             arrowprops=dict(facecolor='green', shrink=0.05), horizontalalignment='center')

"""# Add a text box with the best values
textstr = '\n'.join((
    f'Best Accuracy: {accuracy[best_accuracy_idx]:.5f} at threshold {best_accuracy_threshold}',
    f'Best TPR: {TPR[best_tpr_idx]:.5f} at threshold {best_tpr_threshold}',
))

# Place a text box to the right of the plot
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
plt.gcf().text(0.95, 0.5, textstr, fontsize=12, verticalalignment='center', bbox=props)

plt.show()"""
print(f'\nBest Accuracy: {accuracy[best_accuracy_idx]:.5f} at threshold {best_accuracy_threshold}',
    f'\nBest TPR: {TPR[best_tpr_idx]:.5f} at threshold {best_tpr_threshold}\n') 

In [ ]:
compressed_test = {}
for data in test_data:
    for i in client_models:
        model = client_models[i]
        model.eval()
        layer_output = model(torch.Tensor(data))
        try:
            compressed_test[i].append(layer_output)
        except:
            compressed_test[i] = [layer_output]


In [ ]:
best_tpr_threshold = 0.016031181615218663

In [ ]:
def classify_new_data_fed(new_data_point, threshold):
    # Predict the cluster for the new data point
    new_data_cluster = new_kmeans.predict(np.array(new_data_point))
    # Get the centroid of the predicted cluster
    centroid = new_kmeans.cluster_centers_[new_data_cluster]
    # Calculate the Euclidean distance between the new data point and the centroid
    distance = np.sqrt(np.sum((new_data_point - centroid) ** 2))
    # Classify based on the distance
    if distance < threshold:
        return new_data_cluster[0]
    else:
        return False

threshold = best_tpr_threshold


# Iterate through the compressed_mixed dictionary and classify each data point
for model, data_list in compressed_test.items():
    TP,FP,TN,FN = 0, 0, 0, 0
    j = 0
    for data in data_list:
        # Convert the data to NumPy array and detach from computation graph
        data_np = data.detach().numpy()
        data_np = data_np.reshape(1,-1)
        # Convert the data to double data type before passing it to KMeans model
        if classify_new_data_fed(data_np, threshold) == False:
            classification_result = False
        else: 
            classification_result = cluster_labels[classify_new_data_fed(data_np, threshold)]
        if classification_result == False:
            if model == test_data_labels[j]:
                FN += 1
            else:
                TN += 1
        else:
            if classification_result == test_data_labels[j]:
                if model == test_data_labels[j]:
                    TP += 1
                else:
                    FP += 1
            else:
                FN += 1
        j += 1
            


    print()
    # Calculate accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    # Calculate true positive rate (TPR) or sensitivity or recall
    TPR = TP / (TP + FN)

    # Calculate false positive rate (FPR)
    FPR = FP / (FP + TN)

    # Calculate specificity or true negative rate (TNR)
    TNR = TN / (FP + TN)

    # Calculate precision or positive predictive value (PPV)
    PPV = TP / (TP + FP)



    # Print the metrics with up to four decimal places
    print(f"Data from :" +str(model))
    print("TP:", f"{TP}")
    print("TN:", f"{TN}")
    print("FP:", f"{FP}")
    print("FN:", f"{FN}")
    print("Accuracy:", f"{accuracy*100:.4f}")
    print("True Positive Rate (TPR):", f"{TPR:.4f}")
    print("False Positive Rate (FPR):", f"{FPR:.4f}")
    print("True Negative Rate (TNR):", f"{TNR:.4f}")
    print("Positive Predictive Value (PPV):", f"{PPV:.4f}")
    print()

In [ ]:
threshold = best_tpr_threshold
TP,FP,TN,FN = 0,0,0,0

# Iterate through the compressed_mixed dictionary and classify each data point
for model, data_list in compressed_test.items():
    j = 0
    for data in data_list:
        # Convert the data to NumPy array and detach from computation graph
        data_np = data.detach().numpy()
        data_np = data_np.reshape(1,-1)
        # Convert the data to double data type before passing it to KMeans model
        if classify_new_data_fed(data_np, threshold) == False:
            classification_result = False
        else: 
            classification_result = cluster_labels[classify_new_data_fed(data_np, threshold)]
        if classification_result == False:
            if model == test_data_labels[j]:
                FN += 1
            else:
                TN += 1
        else:
            if classification_result == test_data_labels[j]:
                if model == test_data_labels[j]:
                    TP += 1
                else:
                    FP += 1
            else:
                FN += 1
        j += 1
            


print()
# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate true positive rate (TPR) or sensitivity or recall
TPR = TP / (TP + FN)

# Calculate false positive rate (FPR)
FPR = FP / (FP + TN)

# Calculate specificity or true negative rate (TNR)
TNR = TN / (FP + TN)

# Calculate precision or positive predictive value (PPV)
PPV = TP / (TP + FP)



# Print the metrics with up to four decimal places
print(f"Global Classifier :")
print("TP:", f"{TP}")
print("TN:", f"{TN}")
print("FP:", f"{FP}")
print("FN:", f"{FN}")
print("Accuracy:", f"{accuracy*100:.4f}")
print("True Positive Rate (TPR):", f"{TPR:.4f}")
print("False Positive Rate (FPR):", f"{FPR:.4f}")
print("True Negative Rate (TNR):", f"{TNR:.4f}")
print("Positive Predictive Value (PPV):", f"{PPV:.4f}")
print()

In [ ]:
dump(new_kmeans, '/kaggle/working/models_fixed_overfit/fed_kmeans_model.joblib')